In [13]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import get_dates_datetime, get_dirname, temp_path, get_data_info, update_plate_info, \
get_current_folders, get_folders_by_plate_id


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import run_parallel_transfer, find_state, directory_scratch, directory_project, directory_archive
import dropbox
from amftrack.transfer.functions.transfer import upload, zip_file
from subprocess import call
from tqdm.autonotebook import tqdm
from time import time_ns
import subprocess


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
directory = '/mnt/sun-temp/TEMP/PRINCE_analysis/'
update_plate_info(directory)
all_folders = get_current_folders(directory)
folders = all_folders.loc[all_folders["folder"]=="20220420_0834_Plate12"]

analysed:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp
14286,/mnt/sun-temp/TEMP/PRINCE_analysis/20220420_08...,1019,12,'Carrot','C2','001P100N200C','Y','-',20220419,25,...,False,False,False,False,False,False,False,"20.04.2022, 08:34:",20220420_0834_Plate12,NaN


In [26]:
path_code = os.getenv('HOME')+"/pycode/MscThesis/"

def make_stitching_loop(directory,dirname,op_id):
    a_file = open(f'{path_code}amftrack/pipeline/scripts/stitching_loops/stitching_loop.ijm',"r")

    list_of_lines = a_file.readlines()

    list_of_lines[4] = f'mainDirectory = \u0022{directory}\u0022 ;\n'
    list_of_lines[29] = f'\t if(startsWith(list[i],\u0022{dirname}\u0022)) \u007b\n'
    file_name = f'{os.getenv("TEMP")}/stitching_loops/stitching_loop{op_id}.ijm'
    a_file = open(file_name, "w")

    a_file.writelines(list_of_lines)

    a_file.close()
    
def run_stitch(directory, folders):
    folder_list = list(folders["folder"])
    folder_list.sort()
    with tqdm(total=len(folder_list), desc="stitched") as pbar:
        for folder in folder_list:
            op_id = time_ns()
            im = imageio.imread(f"{directory}/{folder}/Img/Img_r03_c05.tif")
            for x in range(1, 11):
                for y in range(1, 16):
                    strix = str(x) if x >= 10 else f"0{x}"
                    striy = str(y) if y >= 10 else f"0{y}"
                    path = f"{directory}/{folder}/Img/Img_r{strix}_c{striy}.tif"
                    # print(striy,path,os.path.getsize(path))
                    if not os.path.isfile(path):
                        f = open(path, "w")
                    if os.path.getsize(path) == 0:
                        imageio.imwrite(path, im * 0)
            make_stitching_loop(directory, folder, op_id)
            command = [
                os.getenv("HOME") + "/Fiji.app/ImageJ-linux64",
                "--mem=8000m",
                "--headless",
                "--ij2",
                "--console",
                "-macro",
                f'{os.getenv("TEMP")}/stitching_loops/stitching_loop{op_id}.ijm',
            ]
            print(" ".join(command))
            process = subprocess.run(command)
            pbar.update(1)

In [27]:
run_stitch(directory,folders)

stitched:   0%|          | 0/1 [00:00<?, ?it/s]

/home/ipausers/bisot/Fiji.app/ImageJ-linux64 --mem=8000m --headless --ij2 --console -macro /data/temp/stitching_loops/stitching_loop1650453869232875738.ijm


Java HotSpot(TM) 64-Bit Server VM warning: ignoring option PermSize=128m; support was removed in 8.0
Java HotSpot(TM) 64-Bit Server VM warning: Using incremental CMS is deprecated and will likely be removed in a future release


20220420_0834_Plate12/
(Wed Apr 20 13:31:31 CEST 2022): Stitching the following files:
Image: '/mnt/sun-temp/TEMP/PRINCE_analysis//20220420_0834_Plate12//Img2/Img_r01_c01.tif' Imp: 'null' Offset: (0.0, 0.0)
Image: '/mnt/sun-temp/TEMP/PRINCE_analysis//20220420_0834_Plate12//Img2/Img_r01_c02.tif' Imp: 'null' Offset: (3276.0, 0.0)
Image: '/mnt/sun-temp/TEMP/PRINCE_analysis//20220420_0834_Plate12//Img2/Img_r01_c03.tif' Imp: 'null' Offset: (6552.0, 0.0)
Image: '/mnt/sun-temp/TEMP/PRINCE_analysis//20220420_0834_Plate12//Img2/Img_r01_c04.tif' Imp: 'null' Offset: (9828.0, 0.0)
Image: '/mnt/sun-temp/TEMP/PRINCE_analysis//20220420_0834_Plate12//Img2/Img_r01_c05.tif' Imp: 'null' Offset: (13104.0, 0.0)
Image: '/mnt/sun-temp/TEMP/PRINCE_analysis//20220420_0834_Plate12//Img2/Img_r01_c06.tif' Imp: 'null' Offset: (16380.0, 0.0)
Image: '/mnt/sun-temp/TEMP/PRINCE_analysis//20220420_0834_Plate12//Img2/Img_r01_c07.tif' Imp: 'null' Offset: (19656.0, 0.0)
Image: '/mnt/sun-temp/TEMP/PRINCE_analysis//20220420